In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import datetime

sys.path.append('./db')

import alchemy_con
engine = alchemy_con.get_engine()

## Define Target Patients

patients:
- icustays (19778)
    - first_careunit and last_careunit = MICU or CCU
- diagnoses_icd
  - substring(icd9_code, 1, 3) in (570, 571, 572, 573, 070, 155)

In [ ]:
def get_patients_by(diag_ids, icu_types):
    sql = """
        SELECT *
        FROM patients
        WHERE subject_id IN 
            (SELECT DISTINCT (ICD.subject_id)
             FROM icustays AS ICU
             INNER JOIN 
                 (SELECT *
                  FROM diagnoses_icd
                  WHERE SUBSTRING(icd9_code, 1, 3) 
                  IN ({})) AS ICD
                  ON (ICU.hadm_id = ICD.hadm_id)
                  WHERE first_careunit IN ({}));
    """.format(','.join("'{0}'".format(w) for w in diag_ids),\
               ','.join("'{0}'".format(w) for w in icu_types))
    print(sql)
    return pd.read_sql(sql, engine)

In [ ]:
patients = get_patients_by(['570', '571', '572', '573', '070', '155'], ['MICU', 'CCU'])
patients.head()

In [ ]:
patients.shape

In [ ]:
len(patients.subject_id.unique())

In [ ]:
# if want to expand target patients by diagnoses
def get_patients_by_diagnoses_with_keywords(keywords):
    search_by = '|'.join(keywords)
    print(search_by)
    sql = """
        SELECT *
        FROM diagnoses_icd
        WHERE substring(icd9_code, 1, 3) IN 
            (SELECT DISTINCT substring(icd9_code, 1, 3)
             FROM d_icd_diagnoses
             WHERE LOWER(long_title) SIMILAR TO '%%({})%%');
    """.format(search_by)
    print(sql)
    return pd.read_sql(sql, engine)

In [ ]:
icd9_codes = get_patients_by_diagnoses_with_keywords([' liver ', ' hepatitis '])
print(icd9_codes.shape)
icd9_codes.head()

## static attributes:
- age : datetime is shifted for all patients for de-identification. In order to calculate age of a given patient, need to calculate with admission date.  
비식별화를 위해 나이가 shift되어있기 때문에, admission날짜를 기준으로 나이를 계산
    참조 : https://mimic.physionet.org/tutorials/intro-to-mimic-iii/
- gender, ethnicity

### not limiting to last admission id (hadm_id)
```
SELECT
p.subject_id,
p.dob,
a.hadm_id,
a.admittime,
p.gender,
a.ethnicity,
p.expire_flag,
(ROUND((cast(admittime AS DATE) - cast(dob AS DATE)) / 365.242, 2)) AS admit_age
FROM admissions a
INNER JOIN (
          SELECT *
          FROM patients
          WHERE subject_id 
          IN ( SELECT DISTINCT (ICD.subject_id)
               FROM icustays AS ICU
               INNER JOIN ( SELECT *
                            FROM diagnoses_icd
                            WHERE SUBSTRING(icd9_code, 1, 3) 
                            IN ({}) ) AS ICD
                            ON (ICU.hadm_id = ICD.hadm_id)
                            WHERE first_careunit IN ({}) )
         ) AS p
ON p.subject_id = a.subject_id;
```

In [1]:
def get_patients_attrib_by_last_adm(diag_ids, icu_types):
    sql = """
          SELECT
          p.subject_id,
          p.dob,
          a.hadm_id,
          a.admittime,
          p.gender,
          a.ethnicity,
          p.expire_flag,
          (ROUND((cast(admittime AS DATE) - cast(dob AS DATE)) / 365.242, 2)) AS admit_age
        FROM (SELECT
                aa.subject_id,
                aa.hadm_id,
                aa.admittime,
                aa.ethnicity
              FROM admissions aa
                JOIN (SELECT
                        subject_id,
                        MAX(hadm_id) AS m_hadm
                      FROM admissions
                      GROUP BY subject_id) AS max_sub
                  ON aa.hadm_id = m_hadm) AS a
          INNER JOIN (SELECT *
                      FROM patients
                      WHERE subject_id
                            IN (SELECT DISTINCT (ICD.subject_id)
                                FROM icustays AS ICU
                                  INNER JOIN (SELECT *
                                              FROM diagnoses_icd
                                              WHERE SUBSTRING(icd9_code, 1, 3)
                                                    IN ({})) AS ICD
                                    ON (ICU.hadm_id = ICD.hadm_id)
                                WHERE first_careunit IN ({}))
                     ) AS p
            ON p.subject_id = a.subject_id;
    """.format(','.join("'{0}'".format(w) for w in diag_ids),\
               ','.join("'{0}'".format(w) for w in icu_types))
    print(sql)
    return pd.read_sql(sql, engine)

In [5]:
patients_attr = get_patients_attrib_by_last_adm(['570', '571', '572', '573', '070', '155'], ['MICU', 'CCU'])
patients_attr.head()


          SELECT
          p.subject_id,
          p.dob,
          a.hadm_id,
          a.admittime,
          p.gender,
          a.ethnicity,
          p.expire_flag,
          (ROUND((cast(admittime AS DATE) - cast(dob AS DATE)) / 365.242, 2)) AS admit_age
        FROM (SELECT
                aa.subject_id,
                aa.hadm_id,
                aa.admittime,
                aa.ethnicity
              FROM admissions aa
                JOIN (SELECT
                        subject_id,
                        MAX(hadm_id) AS m_hadm
                      FROM admissions
                      GROUP BY subject_id) AS max_sub
                  ON aa.hadm_id = m_hadm) AS a
          INNER JOIN (SELECT *
                      FROM patients
                      WHERE subject_id
                            IN (SELECT DISTINCT (ICD.subject_id)
                                FROM icustays AS ICU
                                  INNER JOIN (SELECT *
                                    

,subject_id,dob,hadm_id,admittime,gender,ethnicity,expire_flag,admit_age
0,70467,2057-02-09,138979,2117-09-14 21:20:00,M,WHITE,0,60.59
1,26770,2103-07-24,186731,2130-05-20 00:11:00,M,WHITE,0,26.82
2,576,2038-05-24,126705,2126-03-18 22:12:00,M,WHITE,0,87.82
3,292,2046-09-17,179726,2103-09-27 18:28:00,F,UNKNOWN/NOT SPECIFIED,1,57.03
4,8057,2087-04-07,128165,2141-01-10 16:17:00,M,WHITE,1,53.76


In [6]:
patients_attr.shape

(3154, 8)

In [7]:
len(patients_attr.subject_id.unique())

3154

In [8]:
# patients_attr.to_csv('./resource/liver_patients.csv', sep=',')

## Dynamic variables

possible variables
- lab
    - RFT : BUN, GFR, Creatine, 24h U/O
    - Biochem : Glucose, Lactate, PaO<sub>2</sub>/FiO<sub>2</sub>, Sodium
    - LFT : AST, ALT, ALP, PT INR, Albumin, Total Bilirubin
    - Hematologic : WBC count, Platelet
    - Intervention : Ventilator, Vasopressor, Dialysis
    
d_items 
  - BUN, 24hr urine output|cc/kg/hr, Glucose, Lactate, Sodium, PaO2, FiO2
  - AST, ALT, Albumin, Total Bilirubin, Platelet
  - Ventilator, Dialysis

# Chosen d_labitems
|Variable|itemid|normal range|unit|choice
|-|-|-|-|-|
|arterial lactate|50813|(0, 2)|mmol/L|max
|blood albumin|50862|(35, 55)|g/L|min & max|
|AST|50878|(7, 40)|IU/L|max|
|ALT|50861|(5, 35)|IU/L|max|
|ALP|50863|(40, 160)|IU/L|max|
|BUN|51006|(22, 46)|mg/dL|max|
|total bilirubin|50885|(0.2, 1.3)|mg/dL|max|
|platelet|51265|(150k, 400k)|uL|min|
|urine volume|51108|-|-|-|
|urine volume, total|51109|(1k, 2k)|mL/day|-|
|creatinine|50912|(50, 110)|umol/L|-|

### 1. BUN
- there are multiple bun values included in d_items and d_labitems

In [11]:
def get_bun(diag_ids, icu_types):
    sql = """
            SELECT
              label,
              item.itemid,
              dbsource,
              subject_id,
              hadm_id,
              icustay_id,
              charttime,
              storetime,
              cgid,
              value,
              valuenum,
              valueuom,
              warning,
              error,
              resultstatus,
              stopped
            FROM (SELECT
                    label,
                    itemid,
                    dbsource
                  FROM d_items
                  WHERE lower(LABEL) LIKE '%%{}%%') AS item INNER JOIN
              (SELECT *
               FROM chartevents
               WHERE icustay_id IN (
                 SELECT DISTINCT (icustay_id)
                 FROM (SELECT
                         icustays.subject_id,
                         hadm_id,
                         MAX(icustay_id) AS icustay_id
                       FROM icustays
                       WHERE first_careunit IN ({})
                       GROUP BY subject_id, hadm_id)
                   AS ICU
                   INNER JOIN (SELECT *
                               FROM diagnoses_icd
                               WHERE SUBSTRING(icd9_code, 1, 3) IN
                                     ({})) AS ICD
                     ON (ICU.hadm_id = ICD.hadm_id))) AS ICU
                ON ICU.itemid = item.itemid
            ORDER BY icustay_id, charttime;
            """.format('bun', ','.join("'{0}'".format(w) for w in icu_types),\
                      ','.join("'{0}'".format(w) for w in diag_ids))

    print(sql)
    return pd.read_sql(sql, engine)

In [12]:
bun_data = get_bun(['570', '571', '572', '573', '070', '155'], ['MICU', 'CCU'])
bun_data


            SELECT
              label,
              item.itemid,
              dbsource,
              subject_id,
              hadm_id,
              icustay_id,
              charttime,
              storetime,
              cgid,
              value,
              valuenum,
              valueuom,
              warning,
              error,
              resultstatus,
              stopped
            FROM (SELECT
                    label,
                    itemid,
                    dbsource
                  FROM d_items
                  WHERE lower(LABEL) LIKE '%%bun%%') AS item INNER JOIN
              (SELECT *
               FROM chartevents
               WHERE icustay_id IN (
                 SELECT DISTINCT (icustay_id)
                 FROM (SELECT
                         icustays.subject_id,
                         hadm_id,
                         MAX(icustay_id) AS icustay_id
                       FROM icustays
                       WHERE first_careunit IN 

,label,itemid,dbsource,subject_id,hadm_id,icustay_id,charttime,storetime,cgid,value,valuenum,valueuom,warning,error,resultstatus,stopped
0,BUN,225624,metavision,93535,121562,200011,2188-08-06 03:17:00,2188-08-06 04:10:00,20889,50,50.0,mg/dL,1.0,0.0,None,None
1,BUN,225624,metavision,93535,121562,200011,2188-08-06 16:37:00,2188-08-06 17:39:00,20889,45,45.0,mg/dL,1.0,0.0,None,None
2,BUN,225624,metavision,93535,121562,200011,2188-08-07 02:02:00,2188-08-07 05:25:00,20889,42,42.0,mg/dL,1.0,0.0,None,None
3,BUN,1162,carevue,14987,165426,200030,2150-11-13 13:15:00,2150-11-13 14:05:00,15331,36,36.0,None,NaN,NaN,Final,NotStopd
4,BUN (6-20),781,carevue,14987,165426,200030,2150-11-13 13:15:00,2150-11-13 14:05:00,15331,36,36.0,None,NaN,NaN,Final,NotStopd
5,BUN (6-20),781,carevue,14987,165426,200030,2150-11-14 02:46:00,2150-11-14 04:01:00,15331,28,28.0,None,NaN,NaN,Final,NotStopd
6,BUN,1162,carevue,14987,165426,200030,2150-11-14 02:46:00,2150-11-14 04:01:00,15331,28,28.0,None,NaN,NaN,Final,NotStopd
7,BUN (6-20),781,carevue,14987,165426,200030,2150-11-14 15:10:00,2150-11-14 16:53:00,15331,26,26.0,None,NaN,NaN,Final,NotStopd
8,BUN,1162,carevue,14987,165426,200030,2150-11-14 15:10:00,2150-11-14 16:53:00,15331,26,26.0,None,NaN,NaN,Final,NotStopd
9,BUN,1162,carevue,14987,165426,200030,2150-11-15 03:00:00,2150-11-15 04:04:00,15331,23,23.0,None,NaN,NaN,Final,NotStopd


In [14]:
bun_data.shape

(44677, 16)

In [15]:
bun_data.label.unique()

array(['BUN', 'BUN (6-20)'], dtype=object)

In [47]:
bun_data.itemid.unique()

AttributeError: 'DataFrame' object has no attribute 'itemid'

In [16]:
bun_data.value.unique()

array(['50', '45', '42', '36', '28', '26', '23', '21', '16', '19', '18',
       '37', '30', '20', '32', '33', '25', '17', '9', '13', '53', '34',
       '10', '8', '39', '27', '35', '41', '48', '5', '14', '62', '64',
       '65', '66', '71', '73', '70', '75', '81', '90', '92', '94', '101',
       '100', '4', '126', '128', '131', '127', '132', '125', '130', '123',
       '115', '112', '80', '91', '67', '60', '24', '46', '51', '58', '72',
       '38', '84', '93', '99', '98', '104', '113', '106', '110', '6', '7',
       '68', '61', '31', '22', '15', '11', '44', '57', '52', '43', '95',
       '54', '12', '3', '47', '63', '77', '69', '59', '78', '40', '55',
       '83', '87', '56', '49', '29', '88', '1', '86', None, '76', '79',
       '74', '89', '105', '96', '85', '82', '156', '153', '161', '165',
       '171', '135', '109', '121', '118', '134', '97', '107', '124',
       '119', '116', '114', '117', '111', '108', '102', '138', '120',
       '137', '140', '147', '151', '2', '103', '136', '15

In [17]:
bun_data.drop(bun_data.loc[bun_data.value.isnull()].index, inplace=True)
bun_data.shape

(44603, 16)

In [18]:
bun_data.drop(bun_data.loc[bun_data.value.isin(['ERROR', 'no data', '999999'])].index, inplace=True)
bun_data.shape

(44598, 16)

In [ ]:
# bun_data.to_csv('./resource/bun_cleaned.csv', sep=',')

In [25]:
# check value column = valuenum are equal since bun value store numeric values
np.where(bun_data.value.astype(float)!=bun_data.valuenum)

(array([], dtype=int64),)

In [21]:
bun_data.columns

Index(['label', 'itemid', 'dbsource', 'subject_id', 'hadm_id', 'icustay_id',
       'charttime', 'storetime', 'cgid', 'value', 'valuenum', 'valueuom',
       'warning', 'error', 'resultstatus', 'stopped'],
      dtype='object')

In [73]:
bun_data = bun_data[['subject_id', 'hadm_id', 'icustay_id', \
                    'charttime', 'valuenum', 'valueuom']]
bun_data.head()

KeyError: "['valuenum'] not in index"

In [29]:
bun_data.valueuom='mg/dL'
bun_data.head()

/Users/grace/workspace/keras/venv/lib/python3.6/site-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


,subject_id,hadm_id,icustay_id,charttime,storetime,cgid,valuenum,valueuom
0,93535,121562,200011,2188-08-06 03:17:00,2188-08-06 04:10:00,20889,50.0,mg/dL
1,93535,121562,200011,2188-08-06 16:37:00,2188-08-06 17:39:00,20889,45.0,mg/dL
2,93535,121562,200011,2188-08-07 02:02:00,2188-08-07 05:25:00,20889,42.0,mg/dL
3,14987,165426,200030,2150-11-13 13:15:00,2150-11-13 14:05:00,15331,36.0,mg/dL
4,14987,165426,200030,2150-11-13 13:15:00,2150-11-13 14:05:00,15331,36.0,mg/dL


In [34]:
bun_data = bun_data.rename(columns={'valuenum':'BUN', 'valueuom':'BUN_valueuom'})
bun_data.head()

,subject_id,hadm_id,icustay_id,charttime,storetime,cgid,BUN,valueuom
0,93535,121562,200011,2188-08-06 03:17:00,2188-08-06 04:10:00,20889,50.0,mg/dL
1,93535,121562,200011,2188-08-06 16:37:00,2188-08-06 17:39:00,20889,45.0,mg/dL
2,93535,121562,200011,2188-08-07 02:02:00,2188-08-07 05:25:00,20889,42.0,mg/dL
3,14987,165426,200030,2150-11-13 13:15:00,2150-11-13 14:05:00,15331,36.0,mg/dL
4,14987,165426,200030,2150-11-13 13:15:00,2150-11-13 14:05:00,15331,36.0,mg/dL


In [42]:
# choose max value group by icustay_id
bun_data = bun_data.sort_values('BUN', ascending=False)\
                    .drop_duplicates(['icustay_id'])
bun_data.shape

In [45]:
bun_data.reset_index().head()

,index,subject_id,hadm_id,icustay_id,charttime,storetime,cgid,BUN,valueuom
0,15629,76930,164055,236758,2108-05-15 04:44:00,2108-05-15 05:54:00,20889,276.0,mg/dL
1,36345,42135,117105,281609,2127-10-26 05:12:00,2127-10-26 06:21:00,20889,242.0,mg/dL
2,19542,17875,164530,245122,2157-11-29 04:46:00,2157-11-29 05:46:00,15331,241.0,mg/dL
3,13809,98665,145585,231827,2159-05-24 18:47:00,2159-05-24 21:32:00,20889,240.0,mg/dL
4,44169,3764,144705,298933,2140-03-16 06:00:00,2140-03-16 08:06:00,15331,229.0,mg/dL


### 2. urine volume & urine volume, total

## 2. Other exact mathing variables

|Variable|itemid|normal range|unit|choice
|-|-|-|-|-|
|arterial lactate|50813|(0, 2)|mmol/L|max
|blood albumin|50862|(35, 55)|g/L|min & max|
|AST|50878|(7, 40)|IU/L|max|
|ALT|50861|(5, 35)|IU/L|max|
|ALP|50863|(40, 160)|IU/L|max|
|total bilirubin|50885|(0.2, 1.3)|mg/dL|max|
|platelet|51265|(150k, 400k)|uL|min|
|creatinine|50912|(50, 110)|umol/L|-|

In [48]:
def get_variables(itemids, diag_ids, icu_types):
    sql = """
            SELECT
             label,
             item.itemid,
             subject_id,
             hadm_id,
             charttime,
             item.fluid,
             valuenum,
             valueuom
           FROM (SELECT
                   label,
                   itemid,
                   fluid
                 FROM d_labitems
                 WHERE itemid IN ({})) AS item INNER JOIN
             (SELECT *
              FROM labevents
              WHERE hadm_id IN (
                SELECT DISTINCT (ICU.hadm_id)
                FROM (SELECT
                        icustays.subject_id,
                        MAX(hadm_id) AS hadm_id
                      FROM icustays
                      WHERE first_careunit IN ({})
                      GROUP BY subject_id)
                  AS ICU
                  INNER JOIN (SELECT *
                              FROM diagnoses_icd
                              WHERE SUBSTRING(icd9_code, 1, 3) IN
                                    ({})) AS ICD
                    ON (ICU.hadm_id = ICD.hadm_id))) AS ICU
               ON ICU.itemid = item.itemid
           ORDER BY hadm_id, charttime
            """.format(','.join(itemids),\
                       ','.join("'{0}'".format(w) for w in icu_types),\
                       ','.join("'{0}'".format(w) for w in diag_ids))

    print(sql)
    return pd.read_sql(sql, engine)

In [50]:
var_data = get_variables(['50813', '50862', '50878', '50861', '50863', '50885', '51265', '50912'], \
              ['570', '571', '572', '573', '070', '155'],\
              ['MICU', 'CCU'])
var_data.head()


            SELECT
             label,
             item.itemid,
             subject_id,
             hadm_id,
             charttime,
             item.fluid,
             valuenum,
             valueuom
           FROM (SELECT
                   label,
                   itemid,
                   fluid
                 FROM d_labitems
                 WHERE itemid IN (50813,50862,50878,50861,50863,50885,51265,50912)) AS item INNER JOIN
             (SELECT *
              FROM labevents
              WHERE hadm_id IN (
                SELECT DISTINCT (ICU.hadm_id)
                FROM (SELECT
                        icustays.subject_id,
                        MAX(hadm_id) AS hadm_id
                      FROM icustays
                      WHERE first_careunit IN ('MICU','CCU')
                      GROUP BY subject_id)
                  AS ICU
                  INNER JOIN (SELECT *
                              FROM diagnoses_icd
                              WHERE SUBSTRING(icd

,label,itemid,subject_id,hadm_id,charttime,fluid,valuenum,valueuom
0,Platelet Count,51265,54610,100003,2150-04-17 12:30:00,Blood,186.0,K/uL
1,Alanine Aminotransferase (ALT),50861,54610,100003,2150-04-17 12:30:00,Blood,126.0,IU/L
2,Albumin,50862,54610,100003,2150-04-17 12:30:00,Blood,2.6,g/dL
3,Alkaline Phosphatase,50863,54610,100003,2150-04-17 12:30:00,Blood,157.0,IU/L
4,Asparate Aminotransferase (AST),50878,54610,100003,2150-04-17 12:30:00,Blood,260.0,IU/L


In [51]:
var_data.shape

(240269, 8)

In [60]:
var_data.isnull().sum()

label          0
itemid         0
subject_id     0
hadm_id        0
charttime      0
fluid          0
valuenum      88
valueuom       0
dtype: int64

In [99]:
var_data.drop(platelet.loc[var_data.valuenum.isnull()].index, inplace=True)
var_data.shape

(240250, 8)

In [100]:
# var_data.to_csv('./resource/exact_m_vars.csv', sep=',')

In [101]:
var_data.label.unique()

array(['Platelet Count', 'Alanine Aminotransferase (ALT)', 'Albumin',
       'Alkaline Phosphatase', 'Asparate Aminotransferase (AST)',
       'Bilirubin, Total', 'Creatinine', 'Lactate'], dtype=object)

In [102]:
platelet = var_data.loc[var_data.label=='Platelet Count']
platelet.head()

,label,itemid,subject_id,hadm_id,charttime,fluid,valuenum,valueuom
0,Platelet Count,51265,54610,100003,2150-04-17 12:30:00,Blood,186.0,K/uL
7,Platelet Count,51265,54610,100003,2150-04-17 17:41:00,Blood,148.0,K/uL
10,Platelet Count,51265,54610,100003,2150-04-18 02:20:00,Blood,162.0,K/uL
12,Platelet Count,51265,54610,100003,2150-04-18 05:25:00,Blood,162.0,K/uL
22,Platelet Count,51265,54610,100003,2150-04-19 03:32:00,Blood,132.0,K/uL


In [103]:
platelet.shape

(49652, 8)

In [104]:
platelet.describe()

,itemid,subject_id,hadm_id,valuenum
count,49652.0,49652.000000,49652.000000,49652.000000
mean,51265.0,40133.396983,153020.542939,156.718521
std,0.0,29363.324204,29390.648146,132.330057
min,51265.0,4.000000,100003.000000,5.000000
25%,51265.0,14861.250000,127837.000000,64.000000
50%,51265.0,29887.000000,156099.000000,116.000000
75%,51265.0,65176.000000,178850.000000,210.000000
max,51265.0,99923.000000,199929.000000,1207.000000


In [105]:
# choose min value group by hadm_id
platelet = platelet.sort_values('valuenum', ascending=True)\
                    .drop_duplicates(['hadm_id'])
platelet = platelet.reset_index().head()
platelet.shape

(5, 9)

In [106]:
platelet.columns

Index(['index', 'label', 'itemid', 'subject_id', 'hadm_id', 'charttime',
       'fluid', 'valuenum', 'valueuom'],
      dtype='object')

In [107]:
platelet = platelet[['itemid', 'subject_id', 'hadm_id', 'charttime', 'valuenum', 'valueuom']]
platelet = platelet.rename(columns={'valuenum':'platelet', 'valueuom':'platelet_valueuom'})
platelet.head()
platelet.shape

,itemid,subject_id,hadm_id,charttime,platelet,platelet_valueuom
0,51265,3891,140833,2140-07-25 03:00:00,1207.0,K/uL
1,51265,29595,164001,2159-08-06 03:38:00,1185.0,K/uL
2,51265,21553,188632,2184-12-23 22:50:00,1146.0,K/uL
3,51265,24868,196654,2178-11-20 05:36:00,1130.0,K/uL
4,51265,17652,132248,2121-11-24 05:45:00,1110.0,K/uL


(5, 6)

In [125]:
def organize_by_label(labels):
    merged = pd.DataFrame() 
    first = True
    for key in labels.keys():
        print(key)
        sub = var_data.loc[var_data.label.str.find(key)>=0]
        sort_by = labels[key]
        if len(sort_by)==1:
            if sort_by[0]=='min':
                sort_b = True
            else:
                sort_b = False
            sub = sub.sort_values('valuenum', ascending=sort_b)\
                    .drop_duplicates(['hadm_id'])
            sub = sub.reset_index().head()
            sub = sub[['itemid', 'subject_id', 'hadm_id', 'charttime', 'valuenum', 'valueuom']]
            sub = sub.rename(columns={'valuenum':key, 'valueuom':key+'_valueuom'})
            print(sub.shape)
            if first:
                merged = sub
                first = False
            else:
                merged = pd.merge(merged, sub, how='outer')#, columns=['subject_id', 'hadm_id'])
            
        else:
            sort_b = True
        
    return merged

In [126]:
var_df = organize_by_label({'Platelet Count':['min'], \
                     'Alanine Aminotransferase (ALT)':['max'],\
                     'Alkaline Phosphatase':'max',\
                     'Asparate Aminotransferase (AST)':'max',\
                     'Bilirubin, Total':'max',\
                     'Lactate':'max'})

# 'Albumin':['min', 'max'],\
# 'Creatinine':['range','range'],\


Platelet Count
(5, 6)
Alanine Aminotransferase (ALT)
(5, 6)
Alkaline Phosphatase
Asparate Aminotransferase (AST)
Bilirubin, Total
Lactate


In [122]:
var_df.head()

(10, 8)